#Урок 6. Задача lookalike (Positive Unlabeled Learning)

__1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с__ https://archive.ics.uci.edu/ml/datasets.php)






In [1]:
import numpy as np
import pandas as pd

In [2]:
!unzip hotel.csv.zip

Archive:  hotel.csv.zip
  inflating: hotel.csv               


In [130]:
df = pd.read_csv('hotel.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [131]:
# Обзор классов целевой переменной
df.is_canceled.value_counts()

0    75166
1    44224
Name: is_canceled, dtype: int64

In [132]:
target = df['is_canceled']
df = df.drop('is_canceled', axis=1)
df.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,342,2015,July,27,1,0,0,2,0.0,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,737,2015,July,27,1,0,0,2,0.0,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,7,2015,July,27,1,0,1,1,0.0,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,13,2015,July,27,1,0,1,1,0.0,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,14,2015,July,27,1,0,2,2,0.0,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   lead_time                       119390 non-null  int64  
 2   arrival_date_year               119390 non-null  int64  
 3   arrival_date_month              119390 non-null  object 
 4   arrival_date_week_number        119390 non-null  int64  
 5   arrival_date_day_of_month       119390 non-null  int64  
 6   stays_in_weekend_nights         119390 non-null  int64  
 7   stays_in_week_nights            119390 non-null  int64  
 8   adults                          119390 non-null  int64  
 9   children                        119386 non-null  float64
 10  babies                          119390 non-null  int64  
 11  meal                            119390 non-null  object 
 12  country         

In [134]:
# Обзор количественных признаков
df.describe()

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [135]:
# Обзор номинативных признаков
for cat_colname in df.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

hotel

City Hotel      79330
Resort Hotel    40060
Name: hotel, dtype: int64
****************************************************************************************************

arrival_date_month

August       13877
July         12661
May          11791
October      11160
April        11089
June         10939
September    10508
March         9794
February      8068
November      6794
December      6780
January       5929
Name: arrival_date_month, dtype: int64
****************************************************************************************************

meal

BB           92310
HB           14463
SC           10650
Undefined     1169
FB             798
Name: meal, dtype: int64
****************************************************************************************************

country

PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
DJI        1
BWA        1
HND        1
VGB        1
NAM        1
Name: country, Length: 177, dtype: int64
***********

In [136]:
# Обработка пропусков
df.isna().sum()

hotel                                  0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company                           112593
days_in_waiting_

In [137]:
df.children.value_counts()

0.0     110796
1.0       4861
2.0       3652
3.0         76
10.0         1
Name: children, dtype: int64

In [138]:
# Создадим вспомогательный признак и заполним его 0, если пропуска в children нет и 1, когда пропуски есть
df['children_nan'] = 0
df.loc[df['children'].isna(), 'children_nan'] = 1
df.children_nan.value_counts()

0    119386
1         4
Name: children_nan, dtype: int64

In [139]:
df.country.value_counts()

PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
DJI        1
BWA        1
HND        1
VGB        1
NAM        1
Name: country, Length: 177, dtype: int64

In [140]:
# Создадим вспомогательный признак и заполним его 0, если пропуска в country нет и 1, когда пропуски есть
df['country_nan'] = 0
df.loc[df['country'].isna(), 'country_nan'] = 1
df.country_nan.value_counts()

0    118902
1       488
Name: country_nan, dtype: int64

In [141]:
df.agent.value_counts()

9.0      31961
240.0    13922
1.0       7191
14.0      3640
7.0       3539
         ...  
289.0        1
432.0        1
265.0        1
93.0         1
304.0        1
Name: agent, Length: 333, dtype: int64

In [142]:
# Создадим вспомогательный признак и заполним его 0, если пропуска в agent нет и 1, когда пропуски есть
df['agent_nan'] = 0
df.loc[df['agent'].isna(), 'agent_nan'] = 1
df.agent_nan.value_counts()

0    103050
1     16340
Name: agent_nan, dtype: int64

In [143]:
df.company.value_counts()

40.0     927
223.0    784
67.0     267
45.0     250
153.0    215
        ... 
104.0      1
531.0      1
160.0      1
413.0      1
386.0      1
Name: company, Length: 352, dtype: int64

In [144]:
# Создадим вспомогательный признак и заполним его 0, если пропуска в company нет и 1, когда пропуски есть
df['company_nan'] = 0
df.loc[df['company'].isna(), 'company_nan'] = 1
df.company_nan.value_counts()

1    112593
0      6797
Name: company_nan, dtype: int64

In [145]:
df.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,children_nan,country_nan,agent_nan,company_nan
0,Resort Hotel,342,2015,July,27,1,0,0,2,0.0,...,Transient,0.0,0,0,Check-Out,2015-07-01,0,0,1,1
1,Resort Hotel,737,2015,July,27,1,0,0,2,0.0,...,Transient,0.0,0,0,Check-Out,2015-07-01,0,0,1,1
2,Resort Hotel,7,2015,July,27,1,0,1,1,0.0,...,Transient,75.0,0,0,Check-Out,2015-07-02,0,0,1,1
3,Resort Hotel,13,2015,July,27,1,0,1,1,0.0,...,Transient,75.0,0,0,Check-Out,2015-07-02,0,0,0,1
4,Resort Hotel,14,2015,July,27,1,0,2,2,0.0,...,Transient,98.0,0,1,Check-Out,2015-07-03,0,0,0,1


In [146]:
modes = df[['children', 'country', 'agent', 'company']].mode()
modes

,children,country,agent,company
0,0.0,PRT,9.0,40.0


In [147]:
values = {'children': 0.0, 'country': 'PRT', 'agent': 9.0, 'company': 40.0}
df[['children', 'country', 'agent', 'company']] = \
df[['children', 'country', 'agent', 'company']].fillna(value=values)

In [148]:
df.isna().sum()

hotel                             0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces 

In [149]:
# Обработка категориальных признаков, кроме reservation_status_date
cat_list = []
for cat_colname in df.select_dtypes(include='object').columns:
  cat_list.append(cat_colname)
for cat_colname in cat_list[:-1]:
  df = pd.concat([df, pd.get_dummies(df[cat_colname])], axis=1)
df.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,No Deposit,Non Refund,Refundable,Contract,Group,Transient,Transient-Party,Canceled,Check-Out,No-Show
0,Resort Hotel,342,2015,July,27,1,0,0,2,0.0,...,1,0,0,0,0,1,0,0,1,0
1,Resort Hotel,737,2015,July,27,1,0,0,2,0.0,...,1,0,0,0,0,1,0,0,1,0
2,Resort Hotel,7,2015,July,27,1,0,1,1,0.0,...,1,0,0,0,0,1,0,0,1,0
3,Resort Hotel,13,2015,July,27,1,0,1,1,0.0,...,1,0,0,0,0,1,0,0,1,0
4,Resort Hotel,14,2015,July,27,1,0,2,2,0.0,...,1,0,0,0,0,1,0,0,1,0


In [150]:
# Обработка reservation_status_date
df['reservation_status_date'] = \
        (pd.to_datetime(df['reservation_status_date']) - pd.Timestamp('1970-01-01')) \
         // pd.Timedelta('1s')
df['reservation_status_date'].head()

0    1435708800
1    1435708800
2    1435795200
3    1435795200
4    1435881600
Name: reservation_status_date, dtype: int64

In [151]:
df = df.drop(cat_list[:-1], axis=1)
df.shape

(119390, 265)

__2. Обучить любой классификатор (какой вам нравится)__

In [152]:
# Разбиваем выборку на тренировочную и тестовую части и обучаем модель
from sklearn.model_selection import train_test_split

x_data = df
y_data = target

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [153]:
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(max_iter=100, random_state=42)
model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [154]:
# Проверяем качество
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print(f"f1: {f1 * 100.0:.2f}%") 
    roc = roc_auc_score(y_test, y_predict)
    print(f"roc: {roc * 100.0:.2f}%") 
    rec = recall_score(y_test, y_predict, average='binary')
    print(f"recall: {rec * 100.0:.2f}%") 
    prc = precision_score(y_test, y_predict, average='binary')
    print(f"precision: {prc * 100.0:.2f}%" ) 
    return f1, roc, rec, prc

    
f1, roc, rec, prc = evaluate_results(y_test, y_predict)
f1_1 = f1
roc_1 = roc 
rec_1 = rec 
prc_1 = prc 

Classification results:
f1: 100.00%
roc: 100.00%
recall: 100.00%
precision: 100.00%


__3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные примеры (класс 1), а только лишь часть__

In [155]:
# Представим, что нам неизвестны негативы и часть позитивов
mod_data = df.copy()
mod_data['target'] = target
mod_data_1 = mod_data.copy()
mod_data_2 = mod_data.copy()

# get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:, -1].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
perc = 0.25
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 11056/44224 as positives and unlabeling the rest


In [156]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    108334
 1     11056
Name: class_test, dtype: int64


In [157]:
mod_data.head()

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,Refundable,Contract,Group,Transient,Transient-Party,Canceled,Check-Out,No-Show,target,class_test
0,342,2015,27,1,0,0,2,0.0,0,0,...,0,0,0,1,0,0,1,0,0,-1
1,737,2015,27,1,0,0,2,0.0,0,0,...,0,0,0,1,0,0,1,0,0,-1
2,7,2015,27,1,0,1,1,0.0,0,0,...,0,0,0,1,0,0,1,0,0,-1
3,13,2015,27,1,0,1,1,0.0,0,0,...,0,0,0,1,0,0,1,0,0,-1
4,14,2015,27,1,0,2,2,0.0,0,0,...,0,0,0,1,0,0,1,0,0,-1


__4. Применить random negative sampling для построения классификатора в новых условиях__

In [158]:
mod_data = mod_data.sample(frac=1)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(11056, 267) (11056, 267)


In [159]:
model = HistGradientBoostingClassifier(max_iter=100, random_state=42)
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'target']), 
          sample_train['class_test'])

y_predict = model.predict(sample_test.drop(columns=['class_test', 'target']))
evaluate_results(sample_test['target'], y_predict)

f1_2 = f1
roc_2 = roc 
rec_2 = rec 
prc_2 = prc

Classification results:
f1: 99.96%
roc: 99.96%
recall: 99.93%
precision: 100.00%


__5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик)__

In [160]:
cls = [f1_1, roc_1, rec_1, prc_1]
pul = [f1_2, roc_2, rec_2, prc_2]
temp_list = [cls, pul]

final_tab = pd.DataFrame(temp_list, index=['classification', 'PU learning'], 
                         columns=['f1', 'roc', 'rec', 'prc'])
final_tab

,f1,roc,rec,prc
classification,1.0,1.0,1.0,1.0
PU learning,1.0,1.0,1.0,1.0


__6. *Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)__

Увеличим Р до 50 %:

In [161]:
# Представим, что нам неизвестны негативы и часть позитивов
# get the indices of the positives samples
pos_ind = np.where(mod_data_1.iloc[:, -1].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)
# leave 50% of the positives marked
perc = 0.5
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 22112/44224 as positives and unlabeling the rest


In [162]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data_1['class_test'] = -1
mod_data_1.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data_1.iloc[:,-1].value_counts())

target variable:
 -1    97278
 1    22112
Name: class_test, dtype: int64


In [163]:
mod_data_1 = mod_data_1.sample(frac=1)


data_N = mod_data_1[mod_data_1['class_test'] == -1]
data_P = mod_data_1[mod_data_1['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(22112, 267) (22112, 267)


In [164]:
model = HistGradientBoostingClassifier(max_iter=100, random_state=42)
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'target']), 
          sample_train['class_test'])

y_predict = model.predict(sample_test.drop(columns=['class_test', 'target']))
evaluate_results(sample_test['target'], y_predict)

f1_3 = f1
roc_3 = roc 
rec_3 = rec 
prc_3 = prc

Classification results:
f1: 99.99%
roc: 99.99%
recall: 99.97%
precision: 100.00%


Уменьшим Р до 15%:

In [165]:
# Представим, что нам неизвестны негативы и часть позитивов
# get the indices of the positives samples
pos_ind = np.where(mod_data_2.iloc[:, -1].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)
# leave 15% of the positives marked
perc = 0.15
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 6634/44224 as positives and unlabeling the rest


In [166]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data_2['class_test'] = -1
mod_data_2.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data_2.iloc[:,-1].value_counts())

target variable:
 -1    112756
 1      6634
Name: class_test, dtype: int64


In [167]:
mod_data_2 = mod_data_2.sample(frac=1)


data_N = mod_data_2[mod_data_2['class_test'] == -1]
data_P = mod_data_2[mod_data_2['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(6634, 267) (6634, 267)


In [168]:
model = HistGradientBoostingClassifier(max_iter=100, random_state=42)
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'target']), 
          sample_train['class_test'])

y_predict = model.predict(sample_test.drop(columns=['class_test', 'target']))
evaluate_results(sample_test['target'], y_predict)

f1_4 = f1
roc_4 = roc 
rec_4 = rec 
prc_4 = prc

Classification results:
f1: 99.56%
roc: 99.56%
recall: 99.12%
precision: 100.00%


In [172]:
pul_g = [f1_3, roc_3, rec_3, prc_3]
pul_l = [f1_4, roc_4, rec_4, prc_4]
temp_list = [pul_g, pul_l]

final_tab_2 = pd.DataFrame(temp_list, index=['PU learning graiter', 'PU learning lower'], 
                         columns=['f1', 'roc', 'rec', 'prc'])
final_tab_2

,f1,roc,rec,prc
PU learning graiter,1.0,1.0,1.0,1.0
PU learning lower,1.0,1.0,1.0,1.0


__Выводы:__

При увеличении Р качество модели незначительно увеличивается, а при уменьшении Р незначительно уменьшается. В случае увеличения Р речь идет об изменениях в сотые доли процента, а при уменьшении об изменениях в десятые доли процента. Т.е. уменьшается качество модели на порядок сильнее чем увеличивается, хотя в обоих случаях изменения качества модели столь незначительны, что при создании итогового датасета сравнения округленно получаются те же значения качества без каких-либо изменений.